In [122]:
import pandas as pd
import re
df= pd.read_csv("C://Users//hp//AppData//Roaming//Microsoft\Windows\Start Menu//Programs//Anaconda3 (32-bit)//footballresults//results.csv")
df["date"] = pd.to_datetime(df["date"])

In [123]:
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [124]:
df.home_team.value_counts()

Brazil                       594
Argentina                    568
Mexico                       555
Germany                      526
England                      524
                            ... 
Chameria                       1
Romani people                  1
Saint Pierre and Miquelon      1
Western Australia              1
Madrid                         1
Name: home_team, Length: 311, dtype: int64

In [125]:
claassement = pd.read_csv("C://Users//hp//AppData//Roaming//Microsoft//Windows//Start Menu//Programs//Anaconda3 (32-bit)//fifaranking//fifa_ranking-2022-10-06.csv")
claassement["rank_date"] = pd.to_datetime(claassement["rank_date"])
claassement["country_full"] = claassement["country_full"].str.replace("IR Iran", "Iran").str.replace("Korea Republic", "South Korea").str.replace("USA", "United States")
claassement = claassement.set_index(['rank_date']).groupby(['country_full'], group_keys=False).resample('D').first().fillna(method='ffill').reset_index()
df_class = df.merge(claassement[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "home_team"], right_on=["rank_date", "country_full"]).drop(["rank_date", "country_full"], axis=1)

df_class = df_class.merge(claassement[["country_full", "total_points", "previous_points", "rank", "rank_change", "rank_date"]], left_on=["date", "away_team"], right_on=["rank_date", "country_full"], suffixes=("_home", "_away")).drop(["rank_date", "country_full"], axis=1)
df_class[(df_class.home_team == "Morocco") | (df_class.away_team == "Morocco")].tail(10)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,total_points_home,previous_points_home,rank_home,rank_change_home,total_points_away,previous_points_away,rank_away,rank_change_away
22076,2022-01-10,Morocco,Ghana,1,0,African Cup of Nations,Yaoundé,Cameroon,True,1529.93,1525.50,28.0,0.0,1428.97,1428.97,52.0,0.0
22087,2022-01-14,Morocco,Comoros,2,0,African Cup of Nations,Yaoundé,Cameroon,True,1529.93,1525.50,28.0,0.0,1117.65,1117.65,132.0,0.0
22101,2022-01-18,Gabon,Morocco,2,2,African Cup of Nations,Yaoundé,Cameroon,True,1262.00,1262.00,89.0,0.0,1529.93,1525.50,28.0,0.0
22116,2022-01-25,Morocco,Malawi,2,1,African Cup of Nations,Yaoundé,Cameroon,True,1529.93,1525.50,28.0,0.0,1127.21,1127.21,129.0,0.0
22144,2022-01-30,Egypt,Morocco,2,1,African Cup of Nations,Yaoundé,Cameroon,True,1451.77,1449.51,45.0,0.0,1529.93,1525.50,28.0,0.0
22359,2022-06-01,United States,Morocco,3,0,Friendly,Cincinnati,United States,False,1633.72,1643.34,15.0,2.0,1551.88,1547.42,24.0,0.0
22489,2022-06-09,Morocco,South Africa,2,1,African Cup of Nations qualification,Rabat,Morocco,False,1551.88,1547.42,24.0,0.0,1356.01,1358.24,69.0,1.0
22558,2022-06-13,Morocco,Liberia,2,0,African Cup of Nations qualification,Casablanca,Morocco,False,1551.88,1547.42,24.0,0.0,1050.50,1058.28,149.0,4.0
22635,2022-09-23,Morocco,Chile,2,0,Friendly,Barcelona,Spain,True,1558.35,1558.90,23.0,1.0,1514.86,1514.86,29.0,0.0
22714,2022-09-27,Paraguay,Morocco,0,0,Friendly,Sevilla,Spain,True,1440.13,1440.13,50.0,0.0,1558.35,1558.90,23.0,1.0


In [126]:
def affect_points(home, away):
    if home > away:
        return pd.Series([0, 3, 0])
    if home < away:
        return pd.Series([1, 0, 3])
    else:
        return pd.Series([2, 1, 1])

results = df_class.apply(lambda x: affect_points(x["home_score"], x["away_score"]), axis=1)
df_class[["result", "home_team_points", "away_team_points"]] = results

In [127]:

df=df_class

In [128]:
df["rank_dif"] = df["rank_home"] - df["rank_away"]
df["sg"] = df["home_score"] - df["away_score"]
df["points_home_by_rank"] = df["home_team_points"]/df["rank_away"]
df["points_away_by_rank"] = df["away_team_points"]/df["rank_home"]


In [129]:
home_team = df[["date", "home_team", "home_score", "away_score", "rank_home", "rank_away","rank_change_home", "total_points_home", "result", "rank_dif", "points_home_by_rank", "home_team_points"]]

away_team = df[["date", "away_team", "away_score", "home_score", "rank_away", "rank_home","rank_change_away", "total_points_away", "result", "rank_dif", "points_away_by_rank", "away_team_points"]]
home_team.columns = [h.replace("home_", "").replace("_home", "").replace("away_", "suf_").replace("_away", "_suf") for h in home_team.columns]

away_team.columns = [a.replace("away_", "").replace("_away", "").replace("home_", "suf_").replace("_home", "_suf") for a in away_team.columns]


In [130]:
team_stats = home_team.append(away_team)
team_stats_raw = team_stats.copy()

C:\Users\hp\AppData\Local\Temp\ipykernel_5464\3382300979.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  team_stats = home_team.append(away_team)


In [131]:
stats_val = []

for index, row in team_stats.iterrows():
    team = row["team"]
    date = row["date"]
    past_games = team_stats.loc[(team_stats["team"] == team) & (team_stats["date"] < date)].sort_values(by=['date'], ascending=False)
    last5 = past_games.head(5)
    
    goals = past_games["score"].mean()
    goals_l5 = last5["score"].mean()
    
    goals_suf = past_games["suf_score"].mean()
    goals_suf_l5 = last5["suf_score"].mean()
    
    rank = past_games["rank_suf"].mean()
    rank_l5 = last5["rank_suf"].mean()
    
    if len(last5) > 0:
        points = past_games["total_points"].values[0] - past_games["total_points"].values[-1]#qtd de pontos ganhos
        points_l5 = last5["total_points"].values[0] - last5["total_points"].values[-1] 
    else:
        points = 0
        points_l5 = 0
        
    gp = past_games["team_points"].mean()
    gp_l5 = last5["team_points"].mean()
    
    gp_rank = past_games["points_by_rank"].mean()
    gp_rank_l5 = last5["points_by_rank"].mean()
    
    stats_val.append([goals, goals_l5, goals_suf, goals_suf_l5, rank, rank_l5, points, points_l5, gp, gp_l5, gp_rank, gp_rank_l5])

In [136]:
stats_cols = ["goals_mean", "goals_mean_l5", "goals_suf_mean", "goals_suf_mean_l5", "rank_mean", "rank_mean_l5", "points_mean", "points_mean_l5", "game_points_mean", "game_points_mean_l5", "game_points_rank_mean", "game_points_rank_mean_l5"]

stats_df = pd.DataFrame(stats_val, columns=stats_cols)

full_df = pd.concat([team_stats.reset_index(drop=True), stats_df], axis=1, ignore_index=False)

In [137]:
home_team_stats = full_df.iloc[:int(full_df.shape[0]/2),:]
away_team_stats = full_df.iloc[int(full_df.shape[0]/2):,:]
home_team_stats.columns[-12:]
home_team_stats = home_team_stats[home_team_stats.columns[-12:]]
away_team_stats = away_team_stats[away_team_stats.columns[-12:]]
home_team_stats.columns = ['home_'+str(col) for col in home_team_stats.columns]
away_team_stats.columns = ['away_'+str(col) for col in away_team_stats.columns]
match_stats = pd.concat([home_team_stats, away_team_stats.reset_index(drop=True)], axis=1, ignore_index=False)
full_df = pd.concat([df, match_stats.reset_index(drop=True)], axis=1, ignore_index=False)

In [138]:
full_df.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'total_points_home',
       'previous_points_home', 'rank_home', 'rank_change_home',
       'total_points_away', 'previous_points_away', 'rank_away',
       'rank_change_away', 'result', 'home_team_points', 'away_team_points',
       'rank_dif', 'sg', 'points_home_by_rank', 'points_away_by_rank',
       'home_goals_mean', 'home_goals_mean_l5', 'home_goals_suf_mean',
       'home_goals_suf_mean_l5', 'home_rank_mean', 'home_rank_mean_l5',
       'home_points_mean', 'home_points_mean_l5', 'home_game_points_mean',
       'home_game_points_mean_l5', 'home_game_points_rank_mean',
       'home_game_points_rank_mean_l5', 'away_goals_mean',
       'away_goals_mean_l5', 'away_goals_suf_mean', 'away_goals_suf_mean_l5',
       'away_rank_mean', 'away_rank_mean_l5', 'away_points_mean',
       'away_points_mean_l5', 'away_game_points_mean',
       'away_game_points_mean_l5', 'away_

In [139]:
def tournement_friendly(x):
    if x == "Friendly":
        return 1
    else: return 0

full_df["is_friendly"] = full_df["tournament"].apply(lambda x: tournement_friendly(x))

In [140]:
full_df = pd.get_dummies(full_df, columns=["is_friendly"])
full_df.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'total_points_home',
       'previous_points_home', 'rank_home', 'rank_change_home',
       'total_points_away', 'previous_points_away', 'rank_away',
       'rank_change_away', 'result', 'home_team_points', 'away_team_points',
       'rank_dif', 'sg', 'points_home_by_rank', 'points_away_by_rank',
       'home_goals_mean', 'home_goals_mean_l5', 'home_goals_suf_mean',
       'home_goals_suf_mean_l5', 'home_rank_mean', 'home_rank_mean_l5',
       'home_points_mean', 'home_points_mean_l5', 'home_game_points_mean',
       'home_game_points_mean_l5', 'home_game_points_rank_mean',
       'home_game_points_rank_mean_l5', 'away_goals_mean',
       'away_goals_mean_l5', 'away_goals_suf_mean', 'away_goals_suf_mean_l5',
       'away_rank_mean', 'away_rank_mean_l5', 'away_points_mean',
       'away_points_mean_l5', 'away_game_points_mean',
       'away_game_points_mean_l5', 'away_

In [141]:
base_df = full_df[["date", "home_team", "away_team", "rank_home", "rank_away","home_score", "away_score","result", "rank_dif", "rank_change_home", "rank_change_away", 'home_goals_mean',
       'home_goals_mean_l5', 'home_goals_suf_mean', 'home_goals_suf_mean_l5',
       'home_rank_mean', 'home_rank_mean_l5', 'home_points_mean',
       'home_points_mean_l5', 'away_goals_mean', 'away_goals_mean_l5',
       'away_goals_suf_mean', 'away_goals_suf_mean_l5', 'away_rank_mean',
       'away_rank_mean_l5', 'away_points_mean', 'away_points_mean_l5','home_game_points_mean', 'home_game_points_mean_l5',
       'home_game_points_rank_mean', 'home_game_points_rank_mean_l5','away_game_points_mean',
       'away_game_points_mean_l5', 'away_game_points_rank_mean',
       'away_game_points_rank_mean_l5',
       'is_friendly_0', 'is_friendly_1']]

base_df.tail()

,date,home_team,away_team,rank_home,rank_away,home_score,away_score,result,rank_dif,rank_change_home,...,home_game_points_mean,home_game_points_mean_l5,home_game_points_rank_mean,home_game_points_rank_mean_l5,away_game_points_mean,away_game_points_mean_l5,away_game_points_rank_mean,away_game_points_rank_mean_l5,is_friendly_0,is_friendly_1
22726,2022-09-27,Norway,Serbia,36.0,25.0,0,2,1,11.0,0.0,...,1.643963,2.0,0.079918,0.090235,1.615819,2.0,0.047035,0.073887,1,0
22727,2022-09-27,Sweden,Slovenia,20.0,65.0,1,1,2,-45.0,0.0,...,1.677333,0.6,0.065824,0.009231,1.333333,1.0,0.035511,0.029545,1,0
22728,2022-09-27,Kosovo,Cyprus,106.0,108.0,5,1,0,-2.0,0.0,...,1.233333,1.2,0.017072,0.016825,0.943548,1.0,0.019249,0.019652,1,0
22729,2022-09-27,Greece,Northern Ireland,49.0,58.0,3,1,0,-9.0,1.0,...,1.618590,2.4,0.061618,0.028040,1.112450,1.0,0.035517,0.009470,1,0
22730,2022-09-30,Fiji,Solomon Islands,163.0,137.0,1,0,0,26.0,0.0,...,1.418182,1.4,0.008891,0.008868,1.428571,2.0,0.008752,0.012387,1,0


In [142]:
base_df.isna().sum()

date                               0
home_team                          0
away_team                          0
rank_home                          0
rank_away                          0
home_score                         0
away_score                         0
result                             0
rank_dif                           0
rank_change_home                   0
rank_change_away                   0
home_goals_mean                   94
home_goals_mean_l5                94
home_goals_suf_mean               94
home_goals_suf_mean_l5            94
home_rank_mean                    94
home_rank_mean_l5                 94
home_points_mean                   0
home_points_mean_l5                0
away_goals_mean                  107
away_goals_mean_l5               107
away_goals_suf_mean              107
away_goals_suf_mean_l5           107
away_rank_mean                   107
away_rank_mean_l5                107
away_points_mean                   0
away_points_mean_l5                0
h

In [143]:
base_df_no_fg = base_df.dropna()

In [144]:
df = base_df_no_fg

In [145]:
def no_draw(x):
    if x == 2:
        return 1
    else:
        return x
    
df["target"] = df["result"].apply(lambda x: no_draw(x))

C:\Users\hp\AppData\Local\Temp\ipykernel_5464\1906038641.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"] = df["result"].apply(lambda x: no_draw(x))


In [146]:
def create_db(df):
    columns = ["home_team", "away_team", "target", "rank_dif", "home_goals_mean", "home_rank_mean", "away_goals_mean", "away_rank_mean", "home_rank_mean_l5", "away_rank_mean_l5", "home_goals_suf_mean", "away_goals_suf_mean", "home_goals_mean_l5", "away_goals_mean_l5", "home_goals_suf_mean_l5", "away_goals_suf_mean_l5", "home_game_points_rank_mean", "home_game_points_rank_mean_l5", "away_game_points_rank_mean", "away_game_points_rank_mean_l5","is_friendly_0", "is_friendly_1"]
    
    base = df.loc[:, columns]
    base.loc[:, "goals_dif"] = base["home_goals_mean"] - base["away_goals_mean"]
    base.loc[:, "goals_dif_l5"] = base["home_goals_mean_l5"] - base["away_goals_mean_l5"]
    base.loc[:, "goals_suf_dif"] = base["home_goals_suf_mean"] - base["away_goals_suf_mean"]
    base.loc[:, "goals_suf_dif_l5"] = base["home_goals_suf_mean_l5"] - base["away_goals_suf_mean_l5"]
    base.loc[:, "goals_per_ranking_dif"] = (base["home_goals_mean"] / base["home_rank_mean"]) - (base["away_goals_mean"] / base["away_rank_mean"])
    base.loc[:, "dif_rank_agst"] = base["home_rank_mean"] - base["away_rank_mean"]
    base.loc[:, "dif_rank_agst_l5"] = base["home_rank_mean_l5"] - base["away_rank_mean_l5"]
    base.loc[:, "dif_points_rank"] = base["home_game_points_rank_mean"] - base["away_game_points_rank_mean"]
    base.loc[:, "dif_points_rank_l5"] = base["home_game_points_rank_mean_l5"] - base["away_game_points_rank_mean_l5"]
    
    model_df = base[["home_team", "away_team", "target", "rank_dif", "goals_dif", "goals_dif_l5", "goals_suf_dif", "goals_suf_dif_l5", "goals_per_ranking_dif", "dif_rank_agst", "dif_rank_agst_l5", "dif_points_rank", "dif_points_rank_l5", "is_friendly_0", "is_friendly_1"]]
    return model_df

In [147]:
model_db = create_db(df)

In [148]:
model_db.head()

,home_team,away_team,target,rank_dif,goals_dif,goals_dif_l5,goals_suf_dif,goals_suf_dif_l5,goals_per_ranking_dif,dif_rank_agst,dif_rank_agst_l5,dif_points_rank,dif_points_rank_l5,is_friendly_0,is_friendly_1
2,Burkina Faso,Mali,0,28.0,0.0,0.0,0.0,0.0,-0.007459,16.0,16.0,-0.007459,-0.007459,0,1
3,Gabon,Ghana,1,16.0,0.0,0.0,0.0,0.0,-0.004183,28.0,28.0,-0.004183,-0.004183,0,1
10,Uganda,Tanzania,0,12.0,2.0,2.0,-2.0,-2.0,0.026630,-12.0,-12.0,0.037500,0.037500,0,1
18,Togo,Zimbabwe,1,47.0,-1.0,-1.0,-1.0,-1.0,-0.009804,-23.0,-23.0,0.002854,0.002854,1,0
23,Cameroon,Finland,1,-22.0,0.0,0.0,0.0,0.0,0.000000,-39.0,-39.0,0.002622,0.002622,1,0


In [149]:
X = model_db.iloc[:, 3:]
y = model_db[["target"]]

In [150]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1)

In [152]:
import numpy as np
grad_boos = GradientBoostingClassifier(random_state=5)

params = {"learning_rate": [0.01, 0.1, 0.5],
            "min_samples_split": [5, 10],
            "min_samples_leaf": [3, 5],
            "max_depth":[3,5,10],
            "max_features":["sqrt"],
            "n_estimators":[100, 200]
         } 

grad_boos_cv = GridSearchCV(grad_boos, params, cv = 3, n_jobs = -1, verbose = False)

grad_boos_cv.fit(X_train.values, np.ravel(y_train))

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=5),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.5],
                         'max_depth': [3, 5, 10], 'max_features': ['sqrt'],
                         'min_samples_leaf': [3, 5],
                         'min_samples_split': [5, 10],
                         'n_estimators': [100, 200]},
             verbose=False)

In [153]:
preduct_qatar_cup= grad_boos_cv.best_estimator_

In [154]:
preduct_qatar_cup

GradientBoostingClassifier(max_features='sqrt', min_samples_leaf=5,
                           min_samples_split=5, random_state=5)

In [155]:
params_rf = {"max_depth": [20],
                "min_samples_split": [10],
                "max_leaf_nodes": [175],
                "min_samples_leaf": [5],
                "n_estimators": [250],
                 "max_features": ["sqrt"],
                }

rf = RandomForestClassifier(random_state=1)

rf_cv = GridSearchCV(rf, params_rf, cv = 3, n_jobs = -1, verbose = False)

rf_cv.fit(X_train.values, np.ravel(y_train))


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=1), n_jobs=-1,
             param_grid={'max_depth': [20], 'max_features': ['sqrt'],
                         'max_leaf_nodes': [175], 'min_samples_leaf': [5],
                         'min_samples_split': [10], 'n_estimators': [250]},
             verbose=False)

In [160]:
def find_stats(team_1):
#team_1 = "Qatar"
    past_games = team_stats_raw[(team_stats_raw["team"] == team_1)].sort_values("date")
    last5 = team_stats_raw[(team_stats_raw["team"] == team_1)].sort_values("date").tail(5)

    team_1_rank = past_games["rank"].values[-1]
    team_1_goals = past_games.score.mean()
    team_1_goals_l5 = last5.score.mean()
    team_1_goals_suf = past_games.suf_score.mean()
    team_1_goals_suf_l5 = last5.suf_score.mean()
    team_1_rank_suf = past_games.rank_suf.mean()
    team_1_rank_suf_l5 = last5.rank_suf.mean()
    team_1_gp_rank = past_games.points_by_rank.mean()
    team_1_gp_rank_l5 = last5.points_by_rank.mean()

    return [team_1_rank, team_1_goals, team_1_goals_l5, team_1_goals_suf, team_1_goals_suf_l5, team_1_rank_suf, team_1_rank_suf_l5, team_1_gp_rank, team_1_gp_rank_l5]

team_1="France"
find_stats(team_1)

[4.0,
 1.8155080213903743,
 0.8,
 0.7192513368983957,
 1.0,
 38.66577540106952,
 21.8,
 0.16350997310439866,
 0.036564171122994656]

In [165]:
def find_features(team_1, team_2):
    rank_dif = team_1[0] - team_2[0]
    goals_dif = team_1[1] - team_2[1]
    goals_dif_l5 = team_1[2] - team_2[2]
    goals_suf_dif = team_1[3] - team_2[3]
    goals_suf_dif_l5 = team_1[4] - team_2[4]
    goals_per_ranking_dif = (team_1[1]/team_1[5]) - (team_2[1]/team_2[5])
    dif_rank_agst = team_1[5] - team_2[5]
    dif_rank_agst_l5 = team_1[6] - team_2[6]
    dif_gp_rank = team_1[7] - team_2[7]
    dif_gp_rank_l5 = team_1[8] - team_2[8]
    
    return [rank_dif, goals_dif, goals_dif_l5, goals_suf_dif, goals_suf_dif_l5, goals_per_ranking_dif, dif_rank_agst, dif_rank_agst_l5, dif_gp_rank, dif_gp_rank_l5, 1, 0]

In [162]:
playoffs = { "demi-Final": [], "Final": []}

In [178]:
for p in playoffs.keys():
    playoffs[p] = []

actual_round = ""
next_rounds = []
print("======= La preduction World cup QATAR_2022 ========")
for p in playoffs.keys():
    if p =="demi-Final" :
        
        playoffs[p] = [["Argentina", "Croatia"], ["France", "Morocco"]]
        for i in range(0, len(playoffs[p]), 1):
            game = playoffs[p][i]
            home = game[0]
            away = game[1]
            team1 = find_stats(home)
            team2 = find_stats(away)
            features_g1 = find_features(team1, team2)
            features_g2 = find_features(team2, team1)
            probs_g1 = preduct_qatar_cup.predict_proba([features_g1])
            probs_g2 = preduct_qatar_cup.predict_proba([features_g2])
            team_1_prob = (probs_g1[0][0] + probs_g2[0][1])/2
            team_2_prob = (probs_g2[0][0] + probs_g1[0][1])/2
            if actual_round != p:
                print("")
                print("====preduction du %s QATAR 2022"%(p))
                print("")
                print("\n")
    
            
            if team_1_prob < team_2_prob:
                print("%s vs. %s: %s a gagner le match avec une probabilité de %.2f"%(home, away, away, team_2_prob))
                next_rounds.append(away)
            else:
                print("%s vs. %s: %s a gagner le match avec une probabilité de %.2f"%(home, away, home, team_1_prob))
                next_rounds.append(home)
            game.append([team_1_prob, team_2_prob])
            playoffs[p][i] = game
            actual_round = p
    
            
    else:
            
        playoffs[p] = [next_rounds[0], next_rounds[1]]
        game = playoffs[p]
        home = game[0]
        away = game[1]
        team1 = find_stats(home)
        team2 = find_stats(away)
            
        features_g1 = find_features(team1, team2)
        features_g2 = find_features(team2, team1)
            
        probs_g1 = preduct_qatar_cup.predict_proba([features_g1])
        probs_g2 = preduct_qatar_cup.predict_proba([features_g2])
            
        team_1_prob = (probs_g1[0][0] + probs_g2[0][1])/2
        team_2_prob = (probs_g2[0][0] + probs_g1[0][1])/2
        if actual_round != p:
            print("")
            print("====preduction du %s QATAR 2022"%(p))
            print("")
            print("\n")
            
        if team_1_prob < team_2_prob:
            print("%s vs. %s: %s a gagner le match avec une probabilité de %.2f"%(home, away, away, team_2_prob))
            next_rounds.append(away)
            print("")
            print("==========")
            print("ma modèle à predit que %s est le vainqueur de la coupe du monde Qatar_2022"%(away))
            print("\n")
        else:
            print("%s vs. %s: %s a gagner le match avec une probabilité de %.2f"%(home, away, home, team_1_prob))
            next_rounds.append(home)
            game.append([team_1_prob, team_2_prob])
            actual_round = p
            print("")
            print("==========")
            print("ma modèle à predit que %s est le vainqueur de la coupe du monde Qatar_2022"%(home))
            print("\n")

======= La preduction World cup QATAR_2022 ========

====preduction du demi-Final QATAR 2022



Argentina vs. Croatia: Argentina a gagner le match avec une probabilité de 0.67
France vs. Morocco: France a gagner le match avec une probabilité de 0.77

====preduction du Final QATAR 2022



Argentina vs. France: Argentina a gagner le match avec une probabilité de 0.57

ma modèle à predit que Argentina est le vainqueur de la coupe du monde Qatar_2022




In [6]:
print("#### commentaire sur ma modele #####")
print("")
print("")
print("\n")
commentaire="dans ma modele j'est utilisé la classification binaire pour prédire le vainqueur de la Coupe du Monde de la FIFA QATAR 2022. car aprés le nottoyage des donnees il est facile de montrer que le classement FIFA de chaque equipe  depend de plusieurs variables qu on peut les construire de plusieurs façon en manupilantes les deux datasets joint pour construires ses variables par exemples les variables sur les nombres de match joué a domicile et à l exterieur.. ou encore le nombre des points ou but de chaque equipe a domicile et l exterieur..tout ses variables permet de données des iformation et d’analyser ce modèle. de sorte que le modèle prédit entre une victoire de l’équipe à domicile et un match nul une victoire de l’équipe à l’extérieur."
print(commentaire)

#### commentaire sur ma modele #####




dans ma modele j'est utilisé la classification binaire pour prédire le vainqueur de la Coupe du Monde de la FIFA QATAR 2022. car aprés le nottoyage des donnees il est facile de montrer que le classement FIFA de chaque equipe  depend de plusieurs variables qu on peut les construire de plusieurs façon en manupilantes les deux datasets joint pour construires ses variables par exemples les variables sur les nombres de match joué a domicile et à l exterieur.. ou encore le nombre des points ou but de chaque equipe a domicile et l exterieur..tout ses variables permet de données des iformation et d’analyser ce modèle. de sorte que le modèle prédit entre une victoire de l’équipe à domicile et un match nul une victoire de l’équipe à l’extérieur.
